<a href="https://colab.research.google.com/github/Olamilek4n/LEK4N/blob/main/Portfolio_Risk_Analysis_S%26P500_Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Building a portfolio using MPT (Markowitz's Modern Portfolio Theory)**

**Assets: Technology Stocks in the S&P 500**

**Selected Stocks: Microsoft, Apple, and Oracle**

**TimeFrame: 365 Days from Current Date**

In [ ]:
# Importing Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader.data as pdr
import datetime as dt
import yfinance as yft
from scipy.optimize import minimize
import plotly.graph_objects as go

In [ ]:
# Portfolio Performance Using the WT
def PortfolioPerformance(weights, MeanReturns, CovMatrix):
    Returns = np.sum(MeanReturns * weights) * 252  # 252 No of Trading days
    Std = np.sqrt(np.dot(weights.T, np.dot(CovMatrix, weights))) * np.sqrt(252)
    return Returns, Std

In [ ]:
# Using sharpeRatio to Minimize scalar function with Sequential Least Squares Programming Algorithm (SLSQP)
def NegativesharpeRatio(weights, MeanReturns, CovMatrix, RiskFreeRate=0):
    PReturns, PStd = PortfolioPerformance(weights, MeanReturns, CovMatrix)
    return -((PReturns - RiskFreeRate) / PStd)

In [ ]:
# Minimize the Negative sharpeRatio by altering the weights of the portfolio
def MaxsharpeRatio(MeanReturns, CovMatrix, RiskFreeRate=0, ConstrainSet=(0, 1)):
    NumAssets = len(MeanReturns)
    args = (MeanReturns, CovMatrix, RiskFreeRate)
    Constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = ConstrainSet
    bounds = tuple(bound for asset in range(NumAssets))
    result = minimize(NegativesharpeRatio, NumAssets * [1. / NumAssets], args=args,
                      method='SLSQP', bounds=bounds, constraints=Constraints)
    return result

In [ ]:
# Minimize the Portfolio Variance by altering the weights of the portfolio
def MinimizeVariance(MeanReturns, CovMatrix, ConstrainSet=(0, 1)):
    NumAssets = len(MeanReturns)
    args = (MeanReturns, CovMatrix)
    Constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = ConstrainSet
    bounds = tuple(bound for asset in range(NumAssets))
    result = minimize(PortfolioVariance, NumAssets * [1. / NumAssets], args=args,
                      method='SLSQP', bounds=bounds, constraints=Constraints)
    return result

In [ ]:
# Efficient Frontier
def EfficientOpt(MeanReturns, CovMatrix, ReturnTarget, ConstrainSet=(0, 1)):
   # For each return target, optimize portfolio for min var
   NumAssets = len(MeanReturns)
   args = (MeanReturns, CovMatrix)

   Constraints = ({'type': 'eq', 'fun': lambda x: PortfolioReturn(x, MeanReturns, CovMatrix) - ReturnTarget},
                  {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
   bound = ConstrainSet
   bounds = tuple(bound for asset in range(NumAssets))
   EffOpt = minimize(PortfolioVariance, NumAssets * [1. / NumAssets], args=args,
                     method='SLSQP', bounds=bounds, constraints=Constraints)

   return EffOpt

In [ ]:
# Portfolio Variance
def PortfolioVariance(weights, MeanReturns, CovMatrix):
    return PortfolioPerformance(weights, MeanReturns, CovMatrix)[1]

# Portfolio Return
def PortfolioReturn(weights, MeanReturns, CovMatrix):
    return PortfolioPerformance(weights, MeanReturns, CovMatrix)[0]

In [ ]:
def CalculateResults(MeanReturns, CovMatrix, RiskFreeRate=0, ConstrainSet=(0, 1)):
    """
    Calculate key portfolio optimization results including Maximum Sharpe Ratio and Minimum Volatility Portfolio.

    Parameters:
    - MeanReturns (pd.Series): Mean returns of individual assets.
    - CovMatrix (pd.DataFrame): Covariance matrix of asset returns.
    - RiskFreeRate (float, optional): Risk-free rate used for Sharpe Ratio calculations. Default is 0.
    - ConstrainSet (tuple, optional): Constraint set for asset weights optimization. Default is (0, 1).

    Returns:
    - max_sharpe_ratio_returns (float): Annualized return of the Maximum Sharpe Ratio portfolio.
    - max_sharpe_ratio_std (float): Annualized standard deviation of the Maximum Sharpe Ratio portfolio.
    - max_sharpe_ratio_allocation (pd.DataFrame): Asset allocation of the Maximum Sharpe Ratio portfolio.
    - MinVol_returns (float): Annualized return of the Minimum Volatility portfolio.
    - MinVol_std (float): Annualized standard deviation of the Minimum Volatility portfolio.
    - MinVol_allocation (pd.DataFrame): Asset allocation of the Minimum Volatility portfolio.
    - EfficientList (list): List of portfolio standard deviations for the Efficient Frontier.
    - TargetReturns (numpy.ndarray): Array of target annualized returns for the Efficient Frontier.
    """

    # Calculate Maximum Sharpe Ratio Portfolio
    max_sharpe_ratio_portfolio = MaxsharpeRatio(MeanReturns, CovMatrix)
    max_sharpe_ratio_returns, max_sharpe_ratio_std = PortfolioPerformance(max_sharpe_ratio_portfolio.x, MeanReturns, CovMatrix)
    max_sharpe_ratio_allocation = pd.DataFrame(max_sharpe_ratio_portfolio.x, index=MeanReturns.index, columns=['allocation'])
    max_sharpe_ratio_allocation.allocation = [round(i * 100, 0) for i in max_sharpe_ratio_allocation.allocation]

    # Calculate Minimum Volatility Portfolio
    MinVol_portfolio = MinimizeVariance(MeanReturns, CovMatrix)
    MinVol_returns, MinVol_std = PortfolioPerformance(MinVol_portfolio.x, MeanReturns, CovMatrix)
    MinVol_allocation = pd.DataFrame(MinVol_portfolio.x, index=MeanReturns.index, columns=['allocation'])
    MinVol_allocation.allocation = [round(i * 100, 0) for i in MinVol_allocation.allocation]

    # Generate target returns for the Efficient Frontier
    TargetReturns = np.linspace(MinVol_returns, max_sharpe_ratio_returns, 20)

    # Calculate portfolio standard deviations for the Efficient Frontier
    EfficientList = []
    for target in TargetReturns:
        EfficientList.append(EfficientOpt(MeanReturns, CovMatrix, target)['fun'])

    # Format results to percentages and return them
    max_sharpe_ratio_returns, max_sharpe_ratio_std = round(max_sharpe_ratio_returns * 100, 2), round(max_sharpe_ratio_std * 100, 2)
    MinVol_returns, MinVol_std = round(MinVol_returns * 100, 2), round(MinVol_std * 100, 2)

    return max_sharpe_ratio_returns, max_sharpe_ratio_std, max_sharpe_ratio_allocation, MinVol_returns, MinVol_std, MinVol_allocation, EfficientList, TargetReturns

print(CalculateResults(MeanReturns, CovMatrix))

(55.72, 26.26,            allocation
Microsoft         0.0
Apple             0.0
Oracle          100.0, 42.71, 23.38,            allocation
Microsoft        11.0
Apple            34.0
Oracle           55.0, [0.2338196323833489, 0.23390346388743014, 0.23415395149687387, 0.23457056133699314, 0.235152410497691, 0.2358982762577235, 0.23680660881471985, 0.23787554697695124, 0.23910293674999802, 0.24048635196300602, 0.24202311712066368, 0.2437103312095117, 0.24554489296997872, 0.24752352610073788, 0.2496428041055743, 0.2518991788041155, 0.25428900018561207, 0.2568143513441275, 0.2595668054886868, 0.26257275074181874], array([0.42714003, 0.43398576, 0.44083149, 0.44767722, 0.45452295,
       0.46136868, 0.46821441, 0.47506014, 0.48190587, 0.4887516 ,
       0.49559733, 0.50244306, 0.50928879, 0.51613452, 0.52298025,
       0.52982598, 0.53667171, 0.54351744, 0.55036317, 0.5572089 ]))


In [ ]:
def EfGraph(MeanReturns, CovMatrix, RiskFreeRate=0, ConstrainSet=(0, 1)):
    # Return graph showing the Minimum Vol, Maximum SR, and EF
    max_sharpe_ratio_returns, max_sharpe_ratio_std, max_sharpe_ratio_allocation, MinVol_returns, MinVol_std, MinVol_allocation, EfficientList, TargetReturns = CalculateResults(MeanReturns, CovMatrix, RiskFreeRate, ConstrainSet)

    # MaxSR
    MaxsharpeRatio = go.Scatter(
        name='Maximum Sharpe Ratio',
        mode='markers',
        x=[max_sharpe_ratio_std],
        y=[max_sharpe_ratio_returns],
        marker=dict(color='red', size=14, line=dict(width=3, color='black'))
    )

    # MinVol
    MinVol = go.Scatter(
        name='Minimum Volatility',
        mode='markers',
        x=[MinVol_std],
        y=[MinVol_returns],
        marker=dict(color='green', size=14, line=dict(width=3, color='black'))
    )

    # Efficient Frontier
    EffCurve = go.Scatter(
        name='Efficient Frontier',
        mode='lines',
        x=[round(EfStd * 100 * 2) for EfStd in EfficientList],
        y=[round(target * 100, 2) for target in TargetReturns],
        line=dict(color='black', width=4, dash='dashdot')
    )

    data = [MaxsharpeRatio, MinVol, EffCurve]

    layout = go.Layout(
    title='Portfolio Optimization with Markowitz Modern Portfolio Theory',
    yaxis=dict(title='Annualized Return (%)'),
    xaxis=dict(title='Annualized Volatility (%)'),
    showlegend=True,
    legend=dict(
        x=0.75, y=0, traceorder='normal',
        bgcolor='#E2E2E2',
        bordercolor='black',
        borderwidth=2,
        font=dict(
            size=14
        )
    )
)

    fig = go.Figure(data=data, layout=layout)
    return fig.show()

In [ ]:
# Define the start and end dates for data retrieval
EndDate = dt.datetime.now()  # Current Date
StartDate = EndDate - dt.timedelta(days=365)

# Downloading the stock data for selected Technology Stocks in the S&P 500 (Microsoft, Nvidia, Apple, and Oracle)
Microsoft = yft.download('MSFT', start=StartDate, end=EndDate)
Apple = yft.download('AAPL', start=StartDate, end=EndDate)
Oracle = yft.download('ORCL', start=StartDate, end=EndDate)

# Combine the closing prices of stocks into a single DataFrame
StocksData = pd.concat([Microsoft['Close'], Apple['Close'], Oracle['Close']], axis=1)
StocksData.columns = ['Microsoft', 'Apple', 'Oracle']

# Returns as percentage change for the stock data
Returns = StocksData.pct_change()

# Mean Returns
MeanReturns = Returns.mean()

# Covariance Matrix
CovMatrix = Returns.cov()

# Setting weights per Stock -- 30%, 30%, 30%, 40%
weights = np.array([0.3, 0.3, 0.4])

Returns, Std = PortfolioPerformance(weights, MeanReturns, CovMatrix)

print(round(Returns * 100, 2), round(Std * 100, 2))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
39.51 23.87


In [ ]:
# Using the EfGraph to Visualize our data
EfGraph(MeanReturns, CovMatrix, RiskFreeRate=0.02)